* 25 % carga y limpieza
    * Carga con Pandas: pd.read_csv

    * Limpieza de valores error: 
        * hay columnas que tienen un valor '?', por tanto se deben reemplazar por un valor nan.

    * Cambio de tipo de dato: .astype() a numéricos y textos, por ejemplo a categorical y carat a float32 o float16

    * Limpieza de nulos (limpiar valores NaN):
        * Nulos en columnas continuas: mediana, media
        * Nulos en columnas categóricas: moda, un valor fijo

    * Encoding: texto a numérico
        * Uso de la función get_dummies() para encoding one_hot
        * Uso de map para encoding ordinal para la columna cut: 1, 2, 3, 4

* 25 % transformaciones:
    * Uso de función apply:
        * crear una columna price_iva a partir de la columna price que muestre el precio + IVA (21%)

    * Crear una nueva columna volumen combinando: x * y * z

    * Ordenar por dos columnas con sort_values():
        * tipo de corte (cut) y precio (price)

    * Agrupaciones con groupby y visualizarla
        * Agrupar por las 3 que hay de tipo categórico calculando la media, max, min por ejemplo de alguna de las numéricas: price, carat, depth

20 % distribuciones: 

* Outliers: Visualización Q1 y Q3 y calcular límites tukey y filtrar. Sobre la columna precio.

* asimetría, curtosis y transformar datos con logaritmo o raíz cuadrada

* Discretizar la columna precio por barato, medio, caro usando la función pd.cut

30 % visualizaciones:

* Seaborn EDAS:
    * univariantes:
        * histogramas y curvas de densidad
        * boxplot
        * countplot
    * bivariantes y multivariantes
        * scatterplot con hue, con size, con style
        * Calcular correlación con Pandas y mostrarla con seaborn
        * Hacer la correlación en un gráfico de barras para la columna 'price'
        * Crear una pivot table usando como index y columns algunas variables categóricas y como values usar el price y visualizarla con heatmap de seaborn
    * Combinarlas con:
        * hue, style, size, row, col usando un relplot
        * filtro


### Librerías a utilizar

In [87]:
import numpy as np
import pandas as pd
import seaborn as sns

### Carga csv con pandas

In [88]:
df = pd.read_csv("diamonds.csv")
df.head(2)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31


In [90]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53939 non-null  float64
 1   cut      53937 non-null  object 
 2   color    53938 non-null  object 
 3   clarity  53938 non-null  object 
 4   depth    53940 non-null  float64
 5   table    53938 non-null  float64
 6   price    53938 non-null  object 
 7   x        53938 non-null  object 
 8   y        53939 non-null  float64
 9   z        53939 non-null  float64
dtypes: float64(5), object(5)
memory usage: 4.1+ MB


In [ ]:
#df["price"] = df["price"].astype("float")
#mean()

### Limpieza NaN

In [84]:
df.count()

carat      53940
cut        53940
color      53940
clarity    53940
depth      53940
table      53940
price      53940
x          53940
y          53940
z          53940
dtype: int64

In [ ]:
df.describe(include="all").T

In [91]:
df.isna().sum()

carat      1
cut        3
color      2
clarity    2
depth      0
table      2
price      2
x          2
y          1
z          1
dtype: int64

* Para el reemplazo de NaN se utilizará el siguiente critero:
    - carat     media
    - cut       "other"
    - color     "other"
    - clarity   "other"
    - table     media
    - price     "other"
    - x         "other"
    - y         media
    - z         media

In [74]:
#

In [83]:
df["carat"] = df["carat"].fillna(df["carat"].mean())
df["cut"] = df["cut"].ffill()
df["color"] = df["color"].fillna(method="ffill")
df["clarity"] = df["clarity"].fillna(method="ffill")
df["table"] = df["table"].fillna(df["table"].mean())
df["price"] = df["price"].fillna(method="ffill")
df["x"] = df["x"].fillna(method="ffill")
df["y"] = df["y"].fillna(df["y"].mean())
df["z"] = df["z"].fillna(df["z"].mean())

C:\Users\user\AppData\Local\Temp\ipykernel_20416\4118857804.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["color"] = df["color"].fillna(method="ffill")
C:\Users\user\AppData\Local\Temp\ipykernel_20416\4118857804.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["clarity"] = df["clarity"].fillna(method="ffill")
C:\Users\user\AppData\Local\Temp\ipykernel_20416\4118857804.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["price"] = df["price"].fillna(method="ffill")
C:\Users\user\AppData\Local\Temp\ipykernel_20416\4118857804.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["x"] = df["x"].fillna(method="ffill")


In [82]:
df.count()


carat      53940
cut        53940
color      53940
clarity    53940
depth      53940
table      53940
price      53940
x          53940
y          53940
z          53940
dtype: int64

### Limpieza valores error

In [89]:
df.replace('?', np.nan)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.2,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


### Cambio de tipo de datos

In [49]:
#

## Encoding

* Get_dummies()

In [50]:
#

* Encodig Ordinal

In [51]:
#

### Creacion columnas

* price_iva

In [52]:
#

* volumen ( x, y, z)

In [53]:
#

### Orden de columnas


* Tipo de corte

* Precio

### Agrupaciones group_by()

* carac

In [54]:
#

* price

In [55]:
#

* depth

In [56]:
#

## Seaborn EDAS
### Univariantes

* Histograma y curva de densidad

In [57]:
#

* Boxplot

In [58]:
#

* Countplot

In [59]:
#

### Bivariantes y Multivariantes
* Scatterplot

In [60]:
# con hue


In [61]:
# con style


In [62]:
# con size

#### Correlacion
* Calculo con pandas

In [63]:
#

* Mostrar con Seaborn

In [64]:
#

* Combinarlas con filtros hue, style

In [65]:
#

## Outliers
* Visualizacion Q1 y Q3 

In [66]:
#

* Limites Tukey y filtro

In [67]:
#

## Transformacion de datos
* Asimetria

* Curtosis

* Transformacion logaritmo

* Transformacion raiz cuadrada

## Discretizar la columna precio
* barato, medio, caro